In [171]:
import polars as pl
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import make_scorer, root_mean_squared_error
import pandas as pd
import numpy as np
from datetime import datetime

**Table of contents**<a id='toc0_'></a>    
- [About](#toc1_)    
- [Data importation](#toc2_)    
- [Data conversion](#toc3_)    
- [Exploratory Data Analysis](#toc4_)    
- [Drop columns](#toc5_)    
- [Remove features](#toc6_)    
  - [Features with too many nulls](#toc6_1_)    
  - [Features with near zero variance](#toc6_2_)    
- [Create features](#toc7_)    
  - [Add bands](#toc7_1_)    
  - [Create time features](#toc7_2_)    
  - [Check if monotonic](#toc7_3_)    
  - [Add seasons](#toc7_4_)    
- [Train test split](#toc8_)    
- [One Hot Encoding](#toc9_)    
- [Train & evaluate model](#toc10_)    
  - [Quantitative evaluation](#toc10_1_)    
  - [Qualitative evaluation](#toc10_2_)    
- [Make predictions](#toc11_)    
- [Export data](#toc12_)    
- [Charts & Graphs](#toc13_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[About](#toc0_)
This notebook illustrates how to perform time-series modelling using the Craigslist vehicle sales as sample data.

In [172]:
pd.options.display.max_columns = None

# <a id='toc2_'></a>[Data importation](#toc0_)
Import all data without inferring to make data importation faster. Columns will be converted to different data types on demand.

In [173]:
N_ROWS = None
craigslist_vehicles = pl.scan_csv("./data/craigslist_vehicles.csv", n_rows=N_ROWS, infer_schema_length=0)
craigslist_vehicles.head().collect()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,image_url,description,county,state,lat,long,posting_date,removal_date
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""362773""","""7307679724""","""https://abilen…","""abilene""","""https://abilen…","""4500""","""2002.0""","""bmw""","""x5""",null,null,"""gas""","""184000.0""","""clean""","""automatic""",null,null,null,null,null,"""https://images…","""$4,500 Cash 2…",null,"""tx""","""32.401556""","""-99.884713""","""2021-04-16 00:…","""2021-05-02 00:…"
"""362712""","""7311833696""","""https://abilen…","""abilene""","""https://abilen…","""4500""","""2002.0""","""bmw""","""x5""",null,null,"""gas""","""184000.0""","""clean""","""automatic""",null,null,null,null,null,"""https://images…","""$4,500 Cash 2…",null,"""tx""","""32.401556""","""-99.884713""","""2021-04-24 00:…","""2021-04-28 00:…"
"""362722""","""7311441996""","""https://abilen…","""abilene""","""https://abilen…","""4900""","""2006.0""","""toyota""","""camry""","""excellent""","""4 cylinders""","""gas""","""184930.0""","""clean""","""automatic""",null,"""fwd""",null,"""sedan""","""silver""","""https://images…","""2006 TOYOTA CA…",null,"""tx""","""32.453848""","""-99.7879""","""2021-04-23 00:…","""2021-05-25 00:…"
"""362771""","""7307680715""","""https://abilen…","""abilene""","""https://abilen…","""6500""","""2008.0""","""ford""","""expedition""",null,null,"""gas""","""206000.0""","""clean""","""automatic""",null,null,null,null,null,"""https://images…","""$6500.00 2008 …",null,"""tx""","""32.401556""","""-99.884713""","""2021-04-16 00:…","""2021-04-26 00:…"
"""362710""","""7311834578""","""https://abilen…","""abilene""","""https://abilen…","""6500""","""2008.0""","""ford""","""expedition""",null,null,"""gas""","""206000.0""","""clean""","""automatic""",null,null,null,null,null,"""https://images…","""$6500.00 2008 …",null,"""tx""","""32.401556""","""-99.884713""","""2021-04-24 00:…","""2021-05-12 00:…"


In [174]:
# Confirming the date format
craigslist_vehicles.collect().sample(20).select(pl.col("removal_date")).to_series().to_list()

['2021-05-30 00:00:00+00:00',
 '2021-05-08 00:00:00+00:00',
 '2021-05-17 00:00:00+00:00',
 '2021-05-13 00:00:00+00:00',
 '2021-06-07 00:00:00+00:00',
 '2021-06-02 00:00:00+00:00',
 '2021-05-26 00:00:00+00:00',
 '2021-04-30 00:00:00+00:00',
 '2021-05-18 00:00:00+00:00',
 '2021-04-30 00:00:00+00:00',
 '2021-05-06 00:00:00+00:00',
 '2021-05-09 00:00:00+00:00',
 '2021-05-01 00:00:00+00:00',
 '2021-05-16 00:00:00+00:00',
 '2021-05-21 00:00:00+00:00',
 '2021-04-20 00:00:00+00:00',
 '2021-05-04 00:00:00+00:00',
 '2021-04-29 00:00:00+00:00',
 '2021-05-13 00:00:00+00:00',
 '2021-05-16 00:00:00+00:00']

# <a id='toc3_'></a>[Data conversion](#toc0_)
Converting specific columns to different data types.

In [175]:
numeric_cols = ["price", "odometer"]
date_cols = ["posting_date", "removal_date"]

In [176]:
def convert_data(data: pl.LazyFrame, date_columns: list = date_cols, numeric_columns: list = numeric_cols) -> pl.LazyFrame:
    
    for d in date_columns:
        data = data.with_columns(pl.col(d).str.to_datetime(format="%Y-%m-%d %H:%M:%S%z"))
        
    for n in numeric_columns:
        data = data.with_columns(pl.col(n).cast(pl.Float32()))
    
    data = data.sort(by="removal_date", descending=False)
    data = data.drop_nulls(subset="removal_date")
    return data

craigslist_vehicles = convert_data(craigslist_vehicles)
craigslist_vehicles.head().collect()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,image_url,description,county,state,lat,long,posting_date,removal_date
str,str,str,str,str,f32,str,str,str,str,str,str,f32,str,str,str,str,str,str,str,str,str,str,str,str,str,"datetime[μs, UTC]","datetime[μs, UTC]"
"""377527""","""7301601318""","""https://killee…","""killeen / temp…","""https://killee…",6500.0,"""2009.0""","""toyota""","""corolla""",null,null,"""other""",117747.0,"""clean""","""automatic""",null,null,null,"""sedan""",null,"""https://images…","""2009 *Toyota* …",null,"""tx""","""31.124479""","""-97.721092""",2021-04-04 00:00:00 UTC,2021-04-04 00:00:00 UTC
"""315630""","""7301716119""","""https://eastor…","""east oregon""","""https://eastor…",23941.0,"""2019.0""","""dodge""","""challenger""",null,"""6 cylinders""","""gas""",28922.0,"""clean""","""automatic""","""2C3CDZAG9KH705…","""rwd""",null,"""coupe""","""black""","""https://images…","""This 2019 Dodg…",null,"""or""","""44.025215""","""-116.991138""",2021-04-04 00:00:00 UTC,2021-04-04 00:00:00 UTC
"""143131""","""7301597966""","""https://lasall…","""la salle co""","""https://lasall…",40990.0,"""2019.0""","""acura""","""mdx advance pk…","""good""","""6 cylinders""","""gas""",12046.0,"""clean""","""other""","""5J8YD3H87KL002…","""fwd""",null,"""other""","""red""","""https://images…","""Carvana is the…",null,"""il""","""33.779214""","""-84.411811""",2021-04-04 00:00:00 UTC,2021-04-04 00:00:00 UTC
"""356810""","""7301671123""","""https://knoxvi…","""knoxville""","""https://knoxvi…",6499.0,"""2009.0""","""saturn""","""outlook""","""excellent""","""6 cylinders""","""gas""",138441.0,"""clean""","""automatic""","""5GZER23D29J185…","""fwd""",null,"""SUV""","""blue""","""https://images…","""Ready To Upgra…",null,"""tn""","""35.898062""","""-84.130619""",2021-04-04 00:00:00 UTC,2021-04-04 00:00:00 UTC
"""81673""","""7301690117""","""https://pueblo…","""pueblo""","""https://pueblo…",22995.0,"""2017.0""","""nissan""","""pathfinder""",null,"""6 cylinders""","""gas""",47202.0,"""clean""","""automatic""","""5N1DR2MM0HC901…","""4wd""",null,"""SUV""","""white""","""https://images…","""This 2017 Niss…",null,"""co""","""38.279325""","""-104.607754""",2021-04-04 00:00:00 UTC,2021-04-04 00:00:00 UTC


In [177]:
min_sale_date = craigslist_vehicles.select("removal_date").min().collect().to_series()[0]

# <a id='toc4_'></a>[Exploratory Data Analysis](#toc0_)
Perform high level Exploratory Data Analysis (EDA).

In [178]:
(craigslist_vehicles
 .collect()
 .describe()
 .transpose(column_names="describe", include_header=True)
)

column,count,null_count,mean,std,min,25%,50%,75%,max
str,str,str,str,str,str,str,str,str,str
"""""","""426812""","""0""",null,null,"""100""",null,null,null,"""99999"""
"""id""","""426812""","""0""",null,null,"""7301583321""",null,null,null,"""7317101084"""
"""url""","""426812""","""0""",null,null,"""https://abilen…",null,null,null,"""https://zanesv…"
"""region""","""426812""","""0""",null,null,"""SF bay area""",null,null,null,"""zanesville / c…"
"""region_url""","""426812""","""0""",null,null,"""https://abilen…",null,null,null,"""https://zanesv…"
"""price""","""426812.0""","""0.0""","""75209.2734375""","""12183253.0""","""0.0""","""5900.0""","""13950.0""","""26489.0""","""3736928768.0"""
"""year""","""425675""","""1137""",null,null,"""1900.0""",null,null,null,"""2022.0"""
"""manufacturer""","""409234""","""17578""",null,null,"""acura""",null,null,null,"""volvo"""
"""model""","""421603""","""5209""",null,null,"""""t""""",null,null,null,"""🔥GMC Sierra 15…"


# <a id='toc5_'></a>[Drop columns](#toc0_)
Drops columns that cannot be used as features. These are columns such as identifiers, urls, text descriptions, etc.

In [179]:
id_cols = ['', "id", "url", "region_url", "VIN", "image_url", "description", "lat", "long", "year"]
craigslist_vehicles = craigslist_vehicles.drop(id_cols)

# <a id='toc6_'></a>[Remove features](#toc0_)
Removes additional columns that cannot be used as features. These are columns such as those that have no variance, with too many null values, or have too many unique values that cannot be consolidated into fewer values.

## <a id='toc6_1_'></a>[Features with too many nulls](#toc0_)
Identify columns that have greater than a certain threshold of null values.

In [180]:
def find_excess_nulls(data: pl.LazyFrame, thr: float = 0.2) -> list:
    df = (data
        .null_count()
        .collect()
        .transpose(include_header=True, column_names=["null_count"])
        .with_columns(pl.lit(value=len(craigslist_vehicles.collect())).alias("obs"))
        .with_columns((pl.col("null_count") / pl.col("obs")).alias("prop"))
        .with_columns((pl.col("prop") > thr).alias("is_excess_nulls"))
        .filter(pl.col("is_excess_nulls") == True)
    )
    
    print(df)
        
    excess_nulls = (df
        .select("column")
        .to_series().to_list()
    )
    
    return excess_nulls

excess_null_cols = find_excess_nulls(craigslist_vehicles)
excess_null_cols

shape: (7, 5)
┌─────────────┬────────────┬────────┬──────────┬─────────────────┐
│ column      ┆ null_count ┆ obs    ┆ prop     ┆ is_excess_nulls │
│ ---         ┆ ---        ┆ ---    ┆ ---      ┆ ---             │
│ str         ┆ u32        ┆ i32    ┆ f64      ┆ bool            │
╞═════════════╪════════════╪════════╪══════════╪═════════════════╡
│ condition   ┆ 174036     ┆ 426812 ┆ 0.407758 ┆ true            │
│ cylinders   ┆ 177610     ┆ 426812 ┆ 0.416132 ┆ true            │
│ drive       ┆ 130499     ┆ 426812 ┆ 0.305753 ┆ true            │
│ size        ┆ 306293     ┆ 426812 ┆ 0.71763  ┆ true            │
│ type        ┆ 92790      ┆ 426812 ┆ 0.217403 ┆ true            │
│ paint_color ┆ 130135     ┆ 426812 ┆ 0.3049   ┆ true            │
│ county      ┆ 426812     ┆ 426812 ┆ 1.0      ┆ true            │
└─────────────┴────────────┴────────┴──────────┴─────────────────┘


['condition', 'cylinders', 'drive', 'size', 'type', 'paint_color', 'county']

## <a id='toc6_2_'></a>[Features with near zero variance](#toc0_)
Identifies columns with little to no variation in their values.

In [181]:
def find_nzv_categorical(data: pl.LazyFrame, thr: float = 0.8) -> list:
    """Identifies categorical columns with a dominant feature that's greater than a certain threshold."""
    
    cols = data.select(pl.col(pl.Utf8)).columns
    
    categorical_cols = [c 
        for c in cols 
        if c not in 
        numeric_cols + excess_null_cols + id_cols
        ]
    
    df = (data
    .select(categorical_cols)
    .melt(variable_name="column")
    .group_by(pl.all())
    .len()
    .rename({"len": "null_count"})
    .with_columns(pl.col("null_count").sum().over("column").alias("total"))
    .with_columns((pl.col("null_count") / pl.col("total")).alias("prop"))
    .with_columns((pl.col("prop") > thr).alias("is_nzv"))
    .sort(by="column")
    .filter(pl.col("is_nzv") == True)
    .collect()
    )
    
    print(df)
    
    is_nzv = (df
    .select("column")
    .to_series().to_list()
    )
    
    return is_nzv

is_nzv_categorical = find_nzv_categorical(craigslist_vehicles)
is_nzv_categorical

shape: (2, 6)
┌──────────────┬───────┬────────────┬────────┬──────────┬────────┐
│ column       ┆ value ┆ null_count ┆ total  ┆ prop     ┆ is_nzv │
│ ---          ┆ ---   ┆ ---        ┆ ---    ┆ ---      ┆ ---    │
│ str          ┆ str   ┆ u32        ┆ u32    ┆ f64      ┆ bool   │
╞══════════════╪═══════╪════════════╪════════╪══════════╪════════╡
│ fuel         ┆ gas   ┆ 356209     ┆ 426812 ┆ 0.834581 ┆ true   │
│ title_status ┆ clean ┆ 405117     ┆ 426812 ┆ 0.94917  ┆ true   │
└──────────────┴───────┴────────────┴────────┴──────────┴────────┘


['fuel', 'title_status']

In [182]:
def find_nzv_numeric(data: pl.LazyFrame, num_cols: list = numeric_cols, thr: float = 0.8) -> list:
    """Identifies numeric columns with little to no variance in their values."""
    
    numeric_data = data.select(num_cols).with_columns(pl.all().cast(pl.Float32()))
    nzv = VarianceThreshold(thr * (1 - thr))
    nzv.fit_transform(numeric_data.collect())
    idx = nzv.get_support(indices=False)
    retained_feats = nzv.get_feature_names_out()[idx]
    return [f for f in numeric_data.columns if f not in retained_feats]

is_nzv_numeric = find_nzv_numeric(craigslist_vehicles)
is_nzv_numeric

[]

In [183]:
# Consolidate all the columns to remove
cols_to_drop = set(id_cols + excess_null_cols + is_nzv_categorical + is_nzv_numeric)
craigslist_vehicles = craigslist_vehicles.drop(cols_to_drop)
craigslist_vehicles.head().collect()

region,price,manufacturer,model,odometer,transmission,state,posting_date,removal_date
str,f32,str,str,f32,str,str,"datetime[μs, UTC]","datetime[μs, UTC]"
"""killeen / temp…",6500.0,"""toyota""","""corolla""",117747.0,"""automatic""","""tx""",2021-04-04 00:00:00 UTC,2021-04-04 00:00:00 UTC
"""east oregon""",23941.0,"""dodge""","""challenger""",28922.0,"""automatic""","""or""",2021-04-04 00:00:00 UTC,2021-04-04 00:00:00 UTC
"""la salle co""",40990.0,"""acura""","""mdx advance pk…",12046.0,"""other""","""il""",2021-04-04 00:00:00 UTC,2021-04-04 00:00:00 UTC
"""knoxville""",6499.0,"""saturn""","""outlook""",138441.0,"""automatic""","""tn""",2021-04-04 00:00:00 UTC,2021-04-04 00:00:00 UTC
"""pueblo""",22995.0,"""nissan""","""pathfinder""",47202.0,"""automatic""","""co""",2021-04-04 00:00:00 UTC,2021-04-04 00:00:00 UTC


# <a id='toc7_'></a>[Create features](#toc0_)
Create specific features of interest.

## <a id='toc7_1_'></a>[Add bands](#toc0_)
Add price and odomoter bands. A better approach here would be to scale the values.

In [184]:
band_cols = ["price", "odometer"]

def create_bucketed_features(data: pl.LazyFrame, cols: list = band_cols, buckets: int = 10) -> pl.LazyFrame:
    
    labels = [str(l) for l in range(buckets)]
    
    for c in cols:
        data = data.with_columns(
            pl.col(c).qcut(quantiles=buckets, labels=labels).cast(pl.Int8()).alias(f"{c}_band"),
            pl.col(c).qcut(quantiles=buckets).alias(f"{c}_band_values")
            )
    
    b_cols = [c for c in data.columns if c.__contains__("_band")]
    v_cols = [c for c in data.columns if c.__contains__("_band_values")]
    
    bands = data.drop(cols + v_cols)
    band_values = data.select(b_cols)
    
    return bands, band_values

craigslist_vehicles_banded, band_values = create_bucketed_features(craigslist_vehicles)
craigslist_vehicles_banded.head().collect()

region,manufacturer,model,transmission,state,posting_date,removal_date,price_band,odometer_band
str,str,str,str,str,"datetime[μs, UTC]","datetime[μs, UTC]",i8,i8
"""lehigh valley""","""honda""","""crv""","""automatic""","""pa""",2021-04-04 00:00:00 UTC,2021-04-04 00:00:00 UTC,0,0
"""augusta""","""lincoln""","""continental re…","""other""","""ga""",2021-04-04 00:00:00 UTC,2021-04-04 00:00:00 UTC,1,1
"""battle creek""","""toyota""","""prius four""","""automatic""","""mi""",2021-04-04 00:00:00 UTC,2021-04-04 00:00:00 UTC,2,2
"""bellingham""","""jeep""","""compass sport …","""automatic""","""wa""",2021-04-04 00:00:00 UTC,2021-04-04 00:00:00 UTC,3,3
"""bellingham""","""chevrolet""","""silverado 2500…","""automatic""","""wa""",2021-04-04 00:00:00 UTC,2021-04-04 00:00:00 UTC,1,2


## <a id='toc7_2_'></a>[Create time features](#toc0_)
Determine how long it has taken to sell a vehicle in terms of years, months, and days.

In [185]:
group_by_cols = craigslist_vehicles_banded.columns + ["year_sold", "month_sold", "day_sold"]
group_by_cols = [c for c in group_by_cols if not c.endswith("date")]

def create_time_features(data: pl.LazyFrame, time_col: str = "removal_date", group_by: list = group_by_cols) -> pl.LazyFrame:
    """Extract the year, month, and day from a specific time column."""
    
    res = (data
        .with_columns(
            pl.col(time_col).dt.year().alias("cal_year"),
            pl.col(time_col).dt.month().alias("month_sold"),
            pl.col(time_col).dt.day().alias("day_sold")
        )
        .with_columns((pl.col("cal_year").min() - pl.col("cal_year")).alias("year_sold"))
        .group_by(pl.col(group_by), maintain_order=True)
        .len()
        .rename({"len": "count"})
    )
    
    return res

model_data = create_time_features(craigslist_vehicles_banded)
model_data.head().collect()

region,manufacturer,model,transmission,state,price_band,odometer_band,year_sold,month_sold,day_sold,count
str,str,str,str,str,i8,i8,i32,i8,i8,u32
"""lehigh valley""","""honda""","""crv""","""automatic""","""pa""",0,0,0,4,4,1
"""augusta""","""lincoln""","""continental re…","""other""","""ga""",1,1,0,4,4,1
"""battle creek""","""toyota""","""prius four""","""automatic""","""mi""",2,2,0,4,4,1
"""bellingham""","""jeep""","""compass sport …","""automatic""","""wa""",3,3,0,4,4,1
"""bellingham""","""chevrolet""","""silverado 2500…","""automatic""","""wa""",1,2,0,4,4,1


## <a id='toc7_3_'></a>[Check if monotonic](#toc0_)
Determine if the data is monotonically increasing.

In [186]:
def create_removal_date(data: pd.DataFrame | pl.LazyFrame) -> list:
    if isinstance(data, pl.LazyFrame):
        data = data.collect().to_pandas()
    
    year_sold = data.year_sold.astype("Int8")
    
    removal_year = [c + min_sale_date.year for c in year_sold]
    removal_month = data.month_sold.astype("Int8")
    removal_day = data.day_sold.astype("Int8")
    
    removal_date = []
    for y, m, d in zip(removal_year, removal_month, removal_day):
        try:
            r = datetime(y, m, d)
            removal_date.append(r)
        except TypeError:
            removal_date.append(None)
    
    return data.assign(removal_date = removal_date)

In [187]:
assert create_removal_date(model_data).removal_date.is_monotonic_increasing

## <a id='toc7_4_'></a>[Add seasons](#toc0_)
Change month of the year into season. This may not be necessary to the model but would be useful for visualisation.

In [188]:
def map_month_to_season(month):
    seasons = {
        "spring": [3, 4, 5],
        "summer": [6, 7, 8],
        "autumn": [9, 10, 11],
        "winter": [12, 1, 2]
    }

    for season, months in seasons.items():
        if month in months:
            return season
        else:
            return "unknown"

def create_seaons(data: pl.LazyFrame) -> pl.LazyFrame:
    res = data.with_columns(
        pl.col("month_sold").map_elements(function=map_month_to_season, skip_nulls=False).alias("season")
    )
    return res

model_data = create_seaons(model_data)
model_data.head().collect()

region,manufacturer,model,transmission,state,price_band,odometer_band,year_sold,month_sold,day_sold,count,season
str,str,str,str,str,i8,i8,i32,i8,i8,u32,str
"""lehigh valley""","""honda""","""crv""","""automatic""","""pa""",0,0,0,4,4,1,"""spring"""
"""augusta""","""lincoln""","""continental re…","""other""","""ga""",1,1,0,4,4,1,"""spring"""
"""battle creek""","""toyota""","""prius four""","""automatic""","""mi""",2,2,0,4,4,1,"""spring"""
"""bellingham""","""jeep""","""compass sport …","""automatic""","""wa""",3,3,0,4,4,1,"""spring"""
"""bellingham""","""chevrolet""","""silverado 2500…","""automatic""","""wa""",1,2,0,4,4,1,"""spring"""


# <a id='toc8_'></a>[Train test split](#toc0_)
Split the data into train/test. Being time series data, we will use scikit-learn's `TimeSeriesSplit`.

In [189]:
y_data = model_data.select("count").collect().to_series()
y = pd.Series(y_data)

X = model_data.drop("count").collect().to_pandas()
print(X.shape)

(422241, 11)


In [ ]:
# We want the forecase for next week
ts_cv = TimeSeriesSplit(n_splits=5, gap=7)
splits = list(ts_cv.split(X, y))

In [191]:
for i, (tr, te) in enumerate(splits):
    tr_len, te_len = len(tr), len(te)
    print(f"split {i} has {tr_len} training indices and {te_len} testing indices: test prop > {te_len/tr_len}")

split 0 has 70369 training indices and 70373 testing indices: 1.00005684321221
split 1 has 140742 training indices and 70373 testing indices: 0.5000142103991702
split 2 has 211115 training indices and 70373 testing indices: 0.3333396490064657
split 3 has 281488 training indices and 70373 testing indices: 0.2500035525493094
split 4 has 351861 training indices and 70373 testing indices: 0.20000227362509626


In [192]:
# split with test proportion of 30%
SPLIT_30_PERC = 2

train_idx, test_idx = splits[SPLIT_30_PERC]
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

In [193]:
# The max date of an obs in train should be less than the min date of an obs in test
print("Max removal date from train")
print(create_removal_date(X_train).removal_date.max())

print("\nMin removal date from test")
print(create_removal_date(X_test).removal_date.min())

assert create_removal_date(X).removal_date.is_monotonic_increasing

Max removal date from train
2021-05-09 00:00:00

Min removal date from test
2021-05-09 00:00:00


# <a id='toc9_'></a>[One Hot Encoding](#toc0_)
Perform One Hot Encoding of the dataset to convert categorical features to numeric.

In [194]:
ohe = OneHotEncoder(drop="if_binary", max_categories=5, dtype=np.int8)
ohe.fit(X)

OneHotEncoder(drop='if_binary', dtype=<class 'numpy.int8'>, max_categories=5)

In [195]:
def ohe_dataframe(data: pd.DataFrame, encoder: OneHotEncoder = ohe) -> pd.DataFrame:
    """One hot encodes the dataframe and labels the columns of the resulting dataframe"""
    
    X = encoder.transform(data)
    X_df = pd.DataFrame(X.toarray(), columns=ohe.get_feature_names_out())
    print("The infrequent categories are:", len(encoder.infrequent_categories_))
    return X_df

X_train_encoded = ohe_dataframe(X_train)
display(X_train_encoded)

The infrequent categories are: 11


,region_columbus,region_jacksonville,region_seattle-tacoma,region_spokane / coeur d'alene,region_infrequent_sklearn,manufacturer_chevrolet,manufacturer_ford,manufacturer_honda,manufacturer_toyota,manufacturer_infrequent_sklearn,model_1500,model_f-150,model_silverado 1500,model_None,model_infrequent_sklearn,transmission_automatic,transmission_manual,transmission_other,transmission_None,state_ca,state_fl,state_ny,state_tx,state_infrequent_sklearn,price_band_0,price_band_6,price_band_7,price_band_8,price_band_infrequent_sklearn,odometer_band_1.0,odometer_band_2.0,odometer_band_7.0,odometer_band_8.0,odometer_band_infrequent_sklearn,year_sold_0,month_sold_4,month_sold_5,month_sold_6,day_sold_9,day_sold_10,day_sold_13,day_sold_14,day_sold_infrequent_sklearn,season_unknown
0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,1,0
1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0
2,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0
3,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,1,0
4,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211110,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0
211111,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0
211112,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0
211113,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0


# <a id='toc10_'></a>[Train & evaluate model](#toc0_)

## <a id='toc10_1_'></a>[Quantitative evaluation](#toc0_)
Trains and evaluates the resulting ML model using the `Root Mean Square Error (RMSE)` evaluation metric.

In [196]:
def train_model(X_train, X_test, y_train, y_test, cv = ts_cv, regressor = HistGradientBoostingRegressor()):
    model = regressor.fit(X_train, y_train)
    
    X_test_encoded = ohe_dataframe(data=X_test)
    y_pred = model.predict(X_test_encoded)
    rmse = root_mean_squared_error(y_true=y_test, y_pred=y_pred)
    
    print("TESTING ERROR:")
    print("rmse:", rmse)
    print("\n")
    
    X_encoded = ohe_dataframe(X)
    scorer = make_scorer(score_func=root_mean_squared_error)
    cv_scores = cross_val_score(estimator=model, X=X_encoded, y=y, scoring=scorer, cv=cv)
    
    print("TESTING ERRORS:")
    print("cv_scores:", cv_scores)
    print("rmse:", cv_scores.mean())
    print("std:", cv_scores.std())

    return model

model = train_model(X_train=X_train_encoded, X_test=X_test, y_train=y_train, y_test=y_test)
model

The infrequent categories are: 11
TESTING ERROR:
rmse: 0.12164732701439507


The infrequent categories are: 11
TESTING ERRORS:
cv_scores: [0.1226668  0.12739058 0.12166833 0.10748986 0.07080129]
rmse: 0.11000337363588035
std: 0.020697197302578376


HistGradientBoostingRegressor()

## <a id='toc10_2_'></a>[Qualitative evaluation](#toc0_)
Qualitative evaluation of the performance of the ML model.

In [197]:
predicted_sales = model.predict(ohe_dataframe(X_test))
print("obs:", len(y_test))
print("total sales (predicted):", sum(predicted_sales))
print("total sales (actual):", sum(y_test))
print("difference:", sum(predicted_sales) - sum(y_test))

The infrequent categories are: 11
obs: 70373
total sales (predicted): 71339.46030877525
total sales (actual): 71289
difference: 50.46030877524754


# <a id='toc11_'></a>[Make predictions](#toc0_)
Prepare data for the dashboard.

In [198]:
predictions = pd.concat(objs=[
    X_test.reset_index(drop=True), 
    pd.Series(y_test, name="actual_sales").reset_index(drop=True),
    pd.Series(predicted_sales, name="predicted_sales"),
    ], axis=1)

predictions

,region,manufacturer,model,transmission,state,price_band,odometer_band,year_sold,month_sold,day_sold,season,actual_sales,predicted_sales
0,norfolk / hampton roads,bmw,435i,manual,va,9,3.0,0,5,9,spring,1,1.008334
1,norfolk / hampton roads,chevrolet,impala limited,automatic,va,6,6.0,0,5,9,spring,1,1.012242
2,norfolk / hampton roads,jeep,wrangler unlimited,automatic,va,9,8.0,0,5,9,spring,1,1.015385
3,norfolk / hampton roads,ram,1500,automatic,va,2,6.0,0,5,9,spring,1,1.033483
4,norfolk / hampton roads,ram,1500,automatic,va,9,4.0,0,5,9,spring,1,1.033483
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70368,clarksville,nissan,frontier crew cab pro-4x,other,tn,7,5.0,0,5,15,spring,1,1.005709
70369,clarksville,dodge,charger,automatic,tn,9,9.0,0,5,15,spring,1,1.014182
70370,clarksville,hyundai,elantra,automatic,tn,9,1.0,0,5,15,spring,1,1.026478
70371,clarksville,volvo,xc90,automatic,tn,5,9.0,0,5,15,spring,1,1.014182


# <a id='toc12_'></a>[Export data](#toc0_)

In [199]:
def extract_band_values(data: pl.LazyFrame, cols: list = band_cols):
    res = {}
    for c in cols:
        r = (data
            .unique(subset=f"{c}_band")
            .select(pl.col([f"{c}_band", f"{c}_band_values"]))
            .filter(pl.col(f"{c}_band").is_not_null())
            .collect()
            .sort(by=f"{c}_band")
        )
        
        res[c]=r
        
    return res

band_values_dict = extract_band_values(band_values)
band_values_dict

{'price': shape: (10, 2)
 ┌────────────┬─────────────────────────────┐
 │ price_band ┆ price_band_values           │
 │ ---        ┆ ---                         │
 │ i8         ┆ cat                         │
 ╞════════════╪═════════════════════════════╡
 │ 0          ┆ (500, 4500]                 │
 │ 1          ┆ (37590, inf]                │
 │ 2          ┆ (17990, 23316.599999999627] │
 │ 3          ┆ (9995, 13950]               │
 │ 4          ┆ (23316.599999999627, 29777] │
 │ 5          ┆ (4500, 6995]                │
 │ 6          ┆ (6995, 9995]                │
 │ 7          ┆ (29777, 37590]              │
 │ 8          ┆ (13950, 17990]              │
 │ 9          ┆ (-inf, 500]                 │
 └────────────┴─────────────────────────────┘,
 'odometer': shape: (10, 2)
 ┌───────────────┬──────────────────────────────┐
 │ odometer_band ┆ odometer_band_values         │
 │ ---           ┆ ---                          │
 │ i8            ┆ cat                          │
 ╞════════

In [200]:
with pd.ExcelWriter(path="./data/predictions.xlsx", mode="w") as writer:
    predictions.to_excel(excel_writer=writer, index=False, sheet_name="predictions")
    for k,v in band_values_dict.items():
        v.to_pandas().to_excel(excel_writer=writer, index=False, sheet_name=k)

# <a id='toc13_'></a>[Charts & Graphs](#toc0_)

* You can be able to explore trends, insights, etc of the model over different 
time spans using [this interactive dashboard](https://lookerstudio.google.com/reporting/2803f46f-1fdf-48d0-8bf7-5c6d6a665bd1/page/xEeoD) that has been published.

* The predictions can be found on [this Google Sheets](https://docs.google.com/spreadsheets/d/1gfdVHUMXRjXx1QRTUdIMxKd-FGWmEJnt9DMn1xrgat8/edit#gid=1316071412).


![alt text](./dashboard_ss.png "Screenshot of Craiglist Vehicle Sales Dashboard")